# Dependencies

In [156]:
!pip install numpy pillow requests

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


## Imports

In [157]:
from PIL import Image
import requests

import os
import re
import csv
import time
from os.path import join, dirname, abspath

from math import cos, sin, pi
from numpy.random import randint
from random import randint, uniform
from numpy import dot, zeros, array, ndarray, uint8

from warnings import warn
from typing import List, Tuple

# Global Configuration

In [158]:
config = {
    "mode": "1",

    "image_stretch_factor": 200,
    "image_tighten_factor": 2,
    "image_size": 200,
    "image_dir": "img_data",

    "mtx_dir": "mtx_data",
    "mtx_max": 1000,
    "mtx_min": 1,

    "save_image_dir": "img_result",
    "save_matrix_dir": "mtx_result",

    "web_site": "https://picsum.photos",

    "verbose": 1,
    "autoclear": 0,
}



# Internal library

In [159]:
def extract_file_name_data(
    file_name: str,
    prefix: str,
    suffix: str
) -> str | None:
    """
    Extracts an identifier given a file name based on a given prefix and suffix

    Args:
        file_name (str): The input file name containing the model name
        prefix (str): The prefix before the model name in the file name
        suffix (str): The suffix after the model name in the file name

    Returns:
        str or None: The extracted model name or None if not found
    """
    pattern = rf"{re.escape(prefix)}([a-zA-Z0-9]+){re.escape(suffix)}"
    match = re.search(pattern, file_name)

    if match:
        return match.group(1)
    else:
        return None


def get_current_path(
    path: str
) -> str:
    """
    Gives the current absolute path

    Args:
        path (str): Path to add into root path

    Returns:
        dir_path (str): Absolute directory path
    """
    current_dir = os.getcwd() 
    dir_path = os.path.join(current_dir, path)

    return dir_path


def file_exists(
    file_path: str
) -> bool:
    """
    Check if a file exists

    Args:
        file_path (str): The path to the file

    Returns:
        bool: True if the file exists, False otherwise
    """
    return os.path.exists(file_path)


def list_files(
    path: str,
    extension: str
) -> List:
    """
    List files with a specific extension in a directory

    Args:
        path (str): The path to the directory containing the files
        extension (str): The file extension to filter files. Only files with
        this extension will be included in the list

    Returns:
        files (list): A list of file names in the specified directory
        (`path`) that have the specified file extension (`extension`)
    """
    files = [f for f in os.listdir(path) if f.endswith(extension)]

    return files


def check_path(
    path: str
) -> bool:
    """
    Checks if a path exist

    Args:
        path (str): Path to check

    Returns:
        True is exist, False if not
    """
    return os.path.isdir(path)


def create_path(
    path: str,
    filename: str
) -> str:
    """
    Creates an absolute path for a file

    Args:
        path (str): A directory path to host the file
        filename (str): A full file name extension included

    Returns:
        file_path (str): A file path to host a file
    """
    absolute_path = get_current_path(path)
    file_path = os.path.join(absolute_path, filename)

    return file_path


def create_dir(
    path: str
) -> None:
    """
    Creates a given path checking is exists of not

    Args:
        path (str): Path to create
    """
    try:
        if check_path(path):
            return
        else:
            dir_path = get_current_path(path)
            os.mkdir(dir_path)

        if config["verbose"]:
            print(f"Directory {path} created")
    except OSError as error_data:
        raise ValueError(f"[ERROR]: Creating {path} \n{error_data}")


def clear_dir(
    path: str
) -> None:
    """
    Delete all files inside a given directory

    Args:
        path (str): Path to directory to clear
    """
    try:
        path = get_current_path(path)
        files = os.listdir(path)
        for file in files:
            file_path = os.path.join(path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)

        if config["verbose"]:
            print(f"Files from {path} deleted successfully.")
    except OSError:
        warn(
            f"[WARNING]: Deleting files at {path}",
            RuntimeWarning,
            stacklevel=1
        )

In [160]:
def create_random(
    size: List[int],
    file_name: str,
    file_extention: str = "jpg"
) -> None:
    """
    Create a file from config given a size and its file name

    Args:
        size (list[int]): List of integers
        file_name (str): File name
        file_extention (str): File extention
    """
    if (len(size) == 1):
        image = f'{config["web_site"]}/{size[0]}'
    else:
        image = f'{config["web_site"]}/{size[0]}/{size[1]}'

    response = requests.get(image)

    if not check_path(config["image_dir"]):
        create_dir(config["image_dir"])

    file_path = create_path(
        config["image_dir"], f"{file_name}.{file_extention}")

    if response.status_code == 200:
        with open(f"{file_path}", "wb") as f:
            f.write(response.content)
        if (config["verbose"]):
            print(f"Image donwloaded at {file_path}")
    else:
        print(f"[ERROR] donwloading image {response.status_code}")


def load_image(
    file_name: str = ""
) -> Image.Image:
    """
    Loads file_name or a random file from config

    Args:
        file_extention (str): Specific image to load

    Returns:
        Image from pillow

    Note:
        Default is random
    """
    try:
        path = get_current_path(config['image_dir'])
        files = os.listdir(path)

        if file_name != "":
            idx = files.index(file_name)
        else:
            idx = randint(0, len(files) - 1)

        file_path = os.path.join(path, files[idx])
        if os.path.isfile(file_path):
            image = Image.open(f"{file_path}")

            return image
    except OSError:
        print(
            f"[ERROR]: Something went wrong while getting image {file_name} or index not found")


def save_image(
    image: Image.Image,
    file_name: str,
    file_extention: str = "PNG"
) -> None:
    """
    Saves a pillow kind image

    Args:
        image (Image): A pillow image
        file_name (str): A specific file name
        file_extention (str): The specific image file extention (Default is jpg)
    """
    try:
        path = get_current_path(config['save_image_dir'])

        if not check_path(f"{config['save_image_dir']}"):
            create_dir(config["save_image_dir"])

        image.save(f"{path}/{file_name}.{file_extention}", f"{file_extention}")
    except OSError:
        print(
            f"[ERROR]: Something went wrong while saving the file {file_name}")

In [161]:
def save_as_csv(data: ndarray, file_name: str) -> None:
    """
    Create a CSV file using a numpy array as data

    Args:
        data (ndarray): A simple numpy array
        file_name (str): Given file name
    """
    if not check_path(config["save_matrix_dir"]):
        create_dir(config["save_matrix_dir"])

    file_path = create_path(config["save_matrix_dir"], f"{file_name}.csv")

    with open(f"{file_path}", mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"')
        for row in data:
            writer.writerow(row)


def load_csv_file() -> List[ndarray]:
    """
    It loads all csv type files from mtx_dir then creates a list of ndarrays

    Returns:
        List of data in ram as ndarray
    """
    csv_data = []
    if not check_path(config["mtx_dir"]):
        warn(
            f"[WARNING]: Directory {config['mtx_dir']} not found. Creating it",
            RuntimeWarning,
            stacklevel=2
        )
        create_dir(config["mtx_dir"])

    path = get_current_path(config['mtx_dir'])
    files = os.listdir(path)

    if (len(files) == 0):
        warn(
            f"[WARNING]: No files provided at {config['mtx_dir']}",
            RuntimeWarning,
            stacklevel=1)

        return csv_data

    for file_name in files:
        file_path = create_path(config["mtx_dir"], file_name)

        with open(f'{file_path}', 'r') as file:
            lines = file.readlines()
            raw_data = []

            for line in lines:
                columns = line.strip().split(',')
                raw_data.append([float(x) for x in columns])

        csv_data.append(array(raw_data))

    return csv_data

# Vectorial calculus

In [162]:
def degrees_to_radians(
    degrees: float
) -> float:
    """
    Convert degrees to radians

    Agrs:
        degrees (int): A number (0 to 360), that means a angle

    Returns:
        Returns the equivalent in radians
    """
    return degrees * (pi / 180.0)


def stretch(
    image: Image.Image,
    stretch_factor: int
) -> Image.Image:
    """
    Stretch an image given a factor casting its byte rate to arrays

    Args:
        image (Image.Image): An image from pillow
        stretch_factor (int): A factor as a integer

    Returns:
        An image casted from an array
    """
    image_array = array(image)
    height, width, dim = image_array.shape

    featured_width = int(width * stretch_factor)
    longed_image = zeros((height, featured_width, dim), dtype=uint8)

    print("shape ", image_array.shape)
    print("shape ", longed_image.shape)

    for y in range(height):
        for x in range(featured_width):
            original_x = int(x / stretch_factor)
            longed_image[y, x] = image_array[y, original_x]

    return Image.fromarray(longed_image)


def tighten(
    image: Image.Image,
    tighten_factor: int
) -> Image.Image:
    """
    Tighten an image given a factor casting its byte rate to arrays

    Args:
        image (Image.Image): An image from pillow
        tighten_factor (int): A factor as a integer

    Returns:
        An image casted from an array
    """
    image_array = array(image)
    height, width, dim = image_array.shape

    featured_height = int(height * tighten_factor)
    tightened_image = zeros((featured_height, width, dim), dtype=uint8)

    for y in range(featured_height):
        for x in range(width):
            original_y = int(y / tighten_factor)
            tightened_image[y, x] = image_array[original_y, x]

    return Image.fromarray(tightened_image)


def rotate_point(
    point: Tuple[float, float],
    angle: float
) -> Tuple[float, float]:
    """
    It rotates a point

    Args:
        point (Tuple[float, float]): A simple par of points
        angle (float): A factor as an angle

    Returns:
        A simple par of points casted
    """
    x, y = point
    x_rotated = x * cos(angle) - y * sin(angle)
    y_rotated = x * sin(angle) + y * cos(angle)
    return x_rotated, y_rotated


def rotate(
    image: Image.Image,
    angle: float
) -> Image.Image:
    """
    It rotates an image given an angle

    Args:
        image (Image.Image): An image object from pillow
        angle (float): A simple factor as an angle

    Returns:
        It rotates an image
    """
    width, height = image.size
    image = image.convert("RGBA")

    rotated_image = Image.new("RGBA", (width, height), (0, 0, 0, 0))
    center_x, center_y = width / 2, height / 2
    pixels = array(image)

    for x in range(width):
        for y in range(height):
            rel_x, rel_y = x - center_x, y - center_y
            rotated_x, rotated_y = rotate_point((rel_x, rel_y), angle)
            rotated_x, rotated_y = rotated_x + center_x, rotated_y + center_y

            if 0 <= rotated_x < width and 0 <= rotated_y < height:
                rotated_image.putpixel(
                    (x, y),
                    tuple(pixels[int(rotated_x), int(rotated_y)])
                )

    return rotated_image

# Matrix calculus

In [163]:
def matrix_verification(
    matrix_A: ndarray,
    matrix_B: ndarray
) -> None:
    """
    Execute a simple verification to check if 2 matrices are compatible

    Args:
        matrix_A (array): Matrix
        matrix_B (array): Matrix

    Note:
        Raise an error to stop execution
    """
    if matrix_A.shape[1] != matrix_B.shape[0]:
        raise ValueError("Matrices are not compatible. Check their dimension")


def matrix_multiplication_check(
    matrix_A: ndarray,
    matrix_B: ndarray
) -> ndarray:
    """
    Test a multiplication between 2 matrices using numpy options

    Args:
        matrix_A (array): A simple matrix
        matrix_B (array): B simple matrix

    Returns:
        It returns the multiplication of those matrices
    """
    dot_test = dot(matrix_A, matrix_B)
    return dot_test


def matrix_multiplication(
    matrix_A: ndarray,
    matrix_B: ndarray
) -> ndarray:
    """
    Multiply 2 matrices without numpy options

    Args:
        matrix_A (array): A matrix as a numpy array
        matrix_B (array): A matrix as a numpy array

    Returns:
        It returns the multiplication of those matrices
    """
    matrix_verification(matrix_A, matrix_B)

    result_matrix = zeros((matrix_A.shape[0], matrix_B.shape[1]))

    for i in range(matrix_A.shape[0]):
        for j in range(matrix_B.shape[1]):
            for k in range(matrix_A.shape[1]):
                result_matrix[i][j] += matrix_A[i][k] * matrix_B[k][j]

    return result_matrix


def load_matrix(schema: List[Tuple[int, int]]):
    """
    Loads an image from "mtx_dir" or load defaults a schema

    Args:
        schema (List[Tuple[int]]): A sizes to build random arrays

    Returns:
        A list of ndarray with data from files at "mtx_dir" or defaults sent
    """
    matrices = []
    files = load_csv_file()

    if len(files) == 0:
        warn(
            "[WARNING]: Loading defaults for matrix",
            RuntimeWarning,
            stacklevel=1
        )

        for sizes in schema:
            matrix = randint(
                config["mtx_min"],
                config["mtx_max"] + 1,
                size=sizes
            )
            matrices.append(matrix)
    else:
        for data in files:
            matrices.append(data)

    return matrices

# Test sets

## Vectorial calculus

In [164]:
def vector_1(timestamp: str) -> None:
    degrees = uniform(0, 360)
    create_random(
        [config["image_size"]],
        f"TV1_d-{str(degrees)}_s-{str(config['image_size'])}_{timestamp}")
    image = load_image()

    angle = degrees_to_radians(degrees)
    moded_image = rotate(image, angle)
    save_image(
        moded_image,
        f"TV1_d-{str(degrees)}_s-{str(config['image_size'])}_{timestamp}")

In [165]:
def vector_2(timestamp: str) -> None:
    factor = config["image_stretch_factor"]
    create_random(
        [config["image_size"]],
        f"TV2_sf-{str(factor)}_{timestamp}")
    image = load_image()

    moded_image = stretch(image, factor)
    save_image(
        moded_image,
        f"TV2_sf-{str(factor)}_{timestamp}")

In [166]:
def vector_3(timestamp: str) -> None:
    factor = config["image_tighten_factor"]
    create_random(
        [config["image_size"]],
        f"TV3_tf-{str(factor)}_{timestamp}")
    image = load_image()

    moded_image = tighten(image, factor)
    save_image(
        moded_image,
        f"TV3_tf-{str(factor)}_{timestamp}")

In [167]:
def vector_4(timestamp: str) -> None:
    degrees = uniform(0, 360)
    angle = degrees_to_radians(degrees)

    tighten_factor = config["image_tighten_factor"]
    stretch_factor = config["image_stretch_factor"]

    create_random([config["image_size"]], f"TV4_s-{str(config['image_size'])}_d-{str(degrees)}_tf-{str(tighten_factor)}_sf-{str(stretch_factor) }_{timestamp}")

    image = load_image()

    moded_image = rotate(image, angle)
    moded_image = stretch(moded_image, stretch_factor)
    moded_image = tighten(moded_image, tighten_factor)

    save_image(moded_image, f"TV4_s-{str(config['image_size'])}_d-{str(degrees)}_tf-{str(tighten_factor)}_sf-{str(stretch_factor)}_{timestamp}")

## Matrix calculus

In [168]:
def matrix_1(timestamp: str) -> None:
    mxA, mxB = load_matrix([(2, 3), (3, 2)])

    if config["verbose"]:
        print("Matrix A:")
        print(mxA)
        print("\n")
        print("Matrix B:")
        print(mxB)

    result = []
    result.append(matrix_multiplication(mxA, mxB))
    result.append(matrix_multiplication_check(mxA, mxB))

    if config["verbose"]:
        print("\n")
        print(f"MTX_1-{timestamp}")
        print("\n")
        print(f"Result: \n{result[0]}")
        print("\n")
        print(f"Verification: \n{result[1]}")

    save_as_csv(mxA, f"MTX_1-mxA-{timestamp}")
    save_as_csv(mxB, f"MTX_1-mxB-{timestamp}")
    save_as_csv(result[0], f"MTX_1-result-{timestamp}")
    save_as_csv(result[1], f"MTX_1-result-{timestamp}")

In [169]:
def matrix_2(timestamp: str) -> None:
    mxA, mxB = load_matrix([(9, 2), (2, 9)])

    if config["verbose"]:
        print("Matrix A:")
        print(mxA)
        print("\n")
        print("Matrix B:")
        print(mxB)

    result = []
    result.append(matrix_multiplication(mxA, mxB))
    result.append(matrix_multiplication_check(mxA, mxB))

    if config["verbose"]:
        print("\n")
        print(f"MTX_2-{timestamp}")
        print("\n")
        print(f"Result: \n{result[0]}")
        print("\n")
        print(f"Verification: \n{result[1]}")

    save_as_csv(mxA, f"MTX_2-mxA-{timestamp}")
    save_as_csv(mxB, f"MTX_2-mxB-{timestamp}")
    save_as_csv(result[0], f"MTX_2-result-{timestamp}")
    save_as_csv(result[1], f"MTX_2-result-{timestamp}")

In [170]:
def matrix_3(timestamp: str) -> None:
    mxA, mxB, mxC = load_matrix([(2, 2), (2, 2), (2, 2)])

    if config["verbose"]:
        print("Matrix A:")
        print(mxA)
        print("\n")
        print("Matrix B:")
        print(mxB)
        print("Matrix C:")
        print(mxC)

    result = []
    result.append(matrix_multiplication(mxA, matrix_multiplication(mxB, mxC)))
    result.append(matrix_multiplication_check(mxA, matrix_multiplication_check(mxB, mxC)))

    if config["verbose"]:
        print("\n")
        print(f"MTX_3-{timestamp}")
        print("\n")
        print(f"Result: \n{result[0]}")
        print("\n")
        print(f"Verification: \n{result[1]}")

    save_as_csv(mxA, f"MTX_3-mxA-{timestamp}")
    save_as_csv(mxB, f"MTX_3-mxB-{timestamp}")
    save_as_csv(result[0], f"MTX_3-result-{timestamp}")
    save_as_csv(result[1], f"MTX_3-result-{timestamp}")

In [171]:
def matrix_4(timestamp: str) -> None:
    mxA, mxB, mxC = load_matrix([(3, 4), (4, 5), (5, 3)])

    if config["verbose"]:
        print("Matrix A:")
        print(mxA)
        print("\n")
        print("Matrix B:")
        print(mxB)
        print("Matrix C:")
        print(mxC)

    result = []
    result.append(matrix_multiplication(mxA, matrix_multiplication(mxB, mxC)))
    result.append(matrix_multiplication_check(mxA, matrix_multiplication_check(mxB, mxC)))

    if config["verbose"]:
        print("\n")
        print(f"MTX_4-{timestamp}")
        print("\n")
        print(f"Result: \n{result[0]}")
        print("\n")
        print(f"Verification: \n{result[1]}")

    save_as_csv(mxA, f"MTX_4-mxA-{timestamp}")
    save_as_csv(mxB, f"MTX_4-mxB-{timestamp}")
    save_as_csv(result[0], f"MTX_4-result-{timestamp}")
    save_as_csv(result[1], f"MTX_4-result-{timestamp}")

In [172]:
test = {
    "1": vector_1,
    "2": vector_2,
    "3": vector_3,
    "4": vector_4,
    "5": matrix_1,
    "6": matrix_2,
    "7": matrix_3,
    "8": matrix_4
}


def select_stage(mode):
    return test.get(mode, "Error")

# Main handler

In [173]:
if __name__ == '__main__':
    if (config["autoclear"]):
        clear_dir(get_current_path(config['mtx_dir']))
        clear_dir(get_current_path(config['image_dir']))
        clear_dir(get_current_path(config['save_image_dir']))
        clear_dir(get_current_path(config['save_matrix_dir']))

    timestamp = str(time.time())

    test = select_stage(config["mode"])

    if test == "Error":
        raise Exception("There is an error on your .env file")

    test(timestamp)



c:\Users\andre\Desktop\math_II\task\vector_and_matrices\img_data
Image donwloaded at c:\Users\andre\Desktop\math_II\task\vector_and_matrices\img_data\TV1_d-211.42613245957196_s-200_1716504805.7455716.jpg
c:\Users\andre\Desktop\math_II\task\vector_and_matrices\img_data
c:\Users\andre\Desktop\math_II\task\vector_and_matrices\img_result
